In [1]:
from sympy import Matrix, Symbol, derive_by_array, Lambda, Function, MatrixSymbol, Derivative, symbols, diff
from sympy import var
from sympy.abc import x, i, j, a, b

In [2]:
def myvar(letter: str, i: int, j: int) -> Symbol:
    letter_ij = Symbol('{}_{}{}'.format(letter, i+1, j+1), is_commutative=True)
    return letter_ij


n,m,p = 3,3,2

X = Matrix(n, m, lambda i,j : myvar('x', i, j)); X

Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]])

In [3]:
W = Matrix(m, p, lambda i,j : myvar('w', i, j)); W

Matrix([
[w_11, w_12],
[w_21, w_22],
[w_31, w_32]])

In [4]:
A = MatrixSymbol('X',3,3); Matrix(A)
B = MatrixSymbol('W',3,2)





In [5]:
v = lambda a,b: a*b

vL = Lambda((a,b), a*b)

vL2 = Lambda((A,B), A*B)

n = Function('v') #, Lambda((a,b), a*b))

vN = lambda mat1, mat2: Matrix(mat1.shape[0], mat2.shape[1], lambda i, j: Symbol("n_{}{}".format(i+1, j+1))); vN


Nelem = vN(X, W)
Nelem

Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])

In [6]:
Nspec = v(X,W)
Nspec

Matrix([
[w_11*x_11 + w_21*x_12 + w_31*x_13, w_12*x_11 + w_22*x_12 + w_32*x_13],
[w_11*x_21 + w_21*x_22 + w_31*x_23, w_12*x_21 + w_22*x_22 + w_32*x_23],
[w_11*x_31 + w_21*x_32 + w_31*x_33, w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [7]:
#N = v(X, W); N
N = n(A,B)
N





v(X, W)

In [8]:
# way 2 of declaring S (better way)
sigma = Function('sigma')

sigmaApply = Function("sigma_apply") #lambda matrix:  matrix.applyfunc(sigma)

sigmaApply_ = lambda matrix: matrix.applyfunc(sigma)


S = sigmaApply(N); S

sigma_apply(v(X, W))

In [9]:
Sspec = S.subs({A:X, B:W}).replace(n, v).replace(sigmaApply, sigmaApply_)
Sspec

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [10]:
Selem = S.replace(n, vN).replace(sigmaApply, sigmaApply_)
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [11]:
import itertools

elemToSpecD = dict(itertools.chain(*[[(Nelem[i, j], Nspec[i, j]) for j in range(2)] for i in range(3)]))

elemToSpec = list(elemToSpecD.items())

Matrix(elemToSpec)

Matrix([
[n_11, w_11*x_11 + w_21*x_12 + w_31*x_13],
[n_12, w_12*x_11 + w_22*x_12 + w_32*x_13],
[n_21, w_11*x_21 + w_21*x_22 + w_31*x_23],
[n_22, w_12*x_21 + w_22*x_22 + w_32*x_23],
[n_31, w_11*x_31 + w_21*x_32 + w_31*x_33],
[n_32, w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [12]:
specToElemD = {v:k for k,v in elemToSpecD.items()}
specToElem = list(specToElemD.items())
Matrix(specToElem)

Matrix([
[w_11*x_11 + w_21*x_12 + w_31*x_13, n_11],
[w_12*x_11 + w_22*x_12 + w_32*x_13, n_12],
[w_11*x_21 + w_21*x_22 + w_31*x_23, n_21],
[w_12*x_21 + w_22*x_22 + w_32*x_23, n_22],
[w_11*x_31 + w_21*x_32 + w_31*x_33, n_31],
[w_12*x_31 + w_22*x_32 + w_32*x_33, n_32]])

In [13]:
elemToSpecFuncD = dict(itertools.chain(*[[(Nelem[i, j], Function("n_{}{}".format(i + 1, j + 1))(Nspec[i, j])) for j in range(2)] for i in range(3)]))

elemToSpecFunc = list(elemToSpecFuncD.items())

Matrix(elemToSpecFunc)

Matrix([
[n_11, n_11(w_11*x_11 + w_21*x_12 + w_31*x_13)],
[n_12, n_12(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[n_21, n_21(w_11*x_21 + w_21*x_22 + w_31*x_23)],
[n_22, n_22(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[n_31, n_31(w_11*x_31 + w_21*x_32 + w_31*x_33)],
[n_32, n_32(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [14]:
elemToSpecFuncArgsD = dict(itertools.chain(*[[(Nelem[i, j], Function("n_{}{}".format(i + 1, j + 1))(*X,*W)) for j in range(2)] for i in range(3)]))

elemToSpecFuncArgs = list(elemToSpecFuncArgsD.items())

Matrix(elemToSpecFuncArgs)

Matrix([
[n_11, n_11(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_12, n_12(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_21, n_21(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_22, n_22(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_31, n_31(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)],
[n_32, n_32(x_11, x_12, x_13, x_21, x_22, x_23, x_31, x_32, x_33, w_11, w_12, w_21, w_22, w_31, w_32)]])

In [15]:
elemToMatArgD = dict(itertools.chain(*[[(Nelem[i, j], Function("n_{}{}".format(i+1,j+1))(A,B) ) for j in range(2)] for i in range(3)]))

elemToMatArg = list(elemToMatArgD.items())

Matrix(elemToMatArg)

Matrix([
[n_11, n_11(X, W)],
[n_12, n_12(X, W)],
[n_21, n_21(X, W)],
[n_22, n_22(X, W)],
[n_31, n_31(X, W)],
[n_32, n_32(X, W)]])

In [16]:
matargToSpecD = dict(zip(elemToMatArgD.values(), elemToSpecD.values()))

matargToSpec = list(matargToSpecD.items())

Matrix(matargToSpec)

Matrix([
[n_11(X, W), w_11*x_11 + w_21*x_12 + w_31*x_13],
[n_12(X, W), w_12*x_11 + w_22*x_12 + w_32*x_13],
[n_21(X, W), w_11*x_21 + w_21*x_22 + w_31*x_23],
[n_22(X, W), w_12*x_21 + w_22*x_22 + w_32*x_23],
[n_31(X, W), w_11*x_31 + w_21*x_32 + w_31*x_33],
[n_32(X, W), w_12*x_31 + w_22*x_32 + w_32*x_33]])

In [17]:
Selem

Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]])

In [18]:
Sspec = Selem.subs(elemToSpecD)
Sspec

Matrix([
[sigma(w_11*x_11 + w_21*x_12 + w_31*x_13), sigma(w_12*x_11 + w_22*x_12 + w_32*x_13)],
[sigma(w_11*x_21 + w_21*x_22 + w_31*x_23), sigma(w_12*x_21 + w_22*x_22 + w_32*x_23)],
[sigma(w_11*x_31 + w_21*x_32 + w_31*x_33), sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)]])

In [19]:
# CAN even replace elements after have done an operation on them!!! replacing n_21 * 2 with the number 4.
Sspec.subs({Nspec[0, 0]: 3}).replace(sigma, lambda x: 2 * x).replace(Nspec[2, 1] * 2, 4)









Matrix([
[                                      6, 2*w_12*x_11 + 2*w_22*x_12 + 2*w_32*x_13],
[2*w_11*x_21 + 2*w_21*x_22 + 2*w_31*x_23, 2*w_12*x_21 + 2*w_22*x_22 + 2*w_32*x_23],
[2*w_11*x_31 + 2*w_21*x_32 + 2*w_31*x_33,                                       4]])

In [20]:
lambd = Function("lambda")
lambd_ = lambda matrix : sum(matrix)

i, j = symbols('i j')
M = MatrixSymbol('M', i, j)# abstract shape
#sigmaApply_L = Lambda(M, M.applyfunc(sigma))

lambda_L = Lambda(M, sum(M))

n = Function("nu",applyfunc=True)

L = lambd(sigmaApply(n(A,B))); L

lambda(sigma_apply(nu(X, W)))

In [21]:
L.replace(n,v).replace(sigmaApply, sigmaApply_).diff(B)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [22]:
L.replace(n,vN).replace(sigmaApply, sigmaApply_)

lambda(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))

In [23]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_)

sigma(n_11) + sigma(n_12) + sigma(n_21) + sigma(n_22) + sigma(n_31) + sigma(n_32)

In [24]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD)

sigma(w_11*x_11 + w_21*x_12 + w_31*x_13) + sigma(w_11*x_21 + w_21*x_22 + w_31*x_23) + sigma(w_11*x_31 + w_21*x_32 + w_31*x_33) + sigma(w_12*x_11 + w_22*x_12 + w_32*x_13) + sigma(w_12*x_21 + w_22*x_22 + w_32*x_23) + sigma(w_12*x_31 + w_22*x_32 + w_32*x_33)

In [25]:
L.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(W).subs(specToElemD)

Matrix([
[x_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32)],
[x_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32)],
[x_13*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_23*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_33*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_13*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_23*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_33*

In [26]:
# Now verifying the above rule by applying the composition thing piece by piece via multiplication:
L.replace(n,vL).replace(sigmaApply, sigmaApply_).diff(B)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [27]:
L.replace(n,vL).replace(sigmaApply,sigmaApply_)#.diff()

lambda(Lambda(_d, sigma(_d)).(X*W))

In [28]:
L.replace(n,vL)

lambda(sigma_apply(X*W))

In [29]:
L.replace(n,vL).diff(sigmaApply(A*B))

Derivative(lambda(sigma_apply(X*W)), sigma_apply(X*W))

In [30]:
#L.replace(n,vL).diff(sigmaApply(A*B)).replace(sigmaApply,sigmaApply_)
## ERROR cannot do that

In [31]:
sigmaApply_L = Lambda(M, M.applyfunc(sigma))
#L.replace(n,vL).diff(sigmaApply(A*B)).subs(sigmaApply,sigmaApply_L) ## ERROR

nL = Lambda((A,B), n(A,B)); nL

Lambda((X, W), nu(X, W))

In [32]:
# Try to create function cpmpositions :
from functools import reduce

def compose2(f, g):
    return lambda *a, **kw: f(g(*a, **kw))

def compose(*fs):
    return reduce(compose2, fs)

In [33]:
f, g, h = symbols("f g h ", cls=Function)
diff(f(g(h(x))), x)

Derivative(f(g(h(x))), g(h(x)))*Derivative(g(h(x)), h(x))*Derivative(h(x), x)

In [34]:
compose(f,g,h)(x)

f(g(h(x)))

In [35]:
compose(f,g,h)

<function __main__.compose2.<locals>.<lambda>(*a, **kw)>

In [36]:
compose(lambd, sigmaApply, n)(A,B)

lambda(sigma_apply(nu(X, W)))

In [37]:
#diff(compose(lambd, sigmaApply, n)(A.T,B), A)


#compose(lambd, sigmaApply, n)(A,B).diff(lambd)
diff(compose(lambd, sigmaApply, n)(A,B), compose(lambd, sigmaApply, n)(A,B))

1

In [38]:
d = diff(compose(lambd, sigmaApply, n)(A,B), n(A,B)); d

Derivative(lambda(sigma_apply(nu(X, W))), sigma_apply(nu(X, W)))*Derivative(sigma_apply(nu(X, W)), nu(X, W))

In [39]:
#compose(lambd, sigmaApply, n)(A,B).diff(A) # ERROR
Lc = compose(lambd, sigmaApply, n)(A, B)
Lc.replace(n, vL).replace(sigmaApply, sigmaApply_)

lambda(Lambda(_d, sigma(_d)).(X*W))

In [40]:
# Same result as replacing in L
#Lc.replace(n, vL).replace(sigmaApply, sigmaApply_).diff(B)
Lc.replace(n, v).replace(sigmaApply, sigmaApply_).diff(B)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [41]:

funcToMat = lambda func_i: Matrix([func_i])
funcToMat(f)
A.applyfunc(sigma)
#funcToMat(f).applyfunc(sigma)
from sympy import FunctionMatrix
F = MatrixSymbol("F",3,3)#(FunctionMatrix(3,3, f))
FL = Lambda(F, n(A,B))
FL
gL = lambda A: A.applyfunc(sigma)
gL(A)
temp = lambda n : n
temp(n(A,B))
#sigmaApply_L(A).subs(A, Lambda((A,B), vL(A,B)))# arg must be matrix instance

nu(X, W)

In [42]:
sigmaApply_L(A*B).diff(B)

X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [43]:
sigmaApply_L(A).diff(A).subs(A,X).doit()

Matrix([
[Derivative(sigma(x_11), x_11), Derivative(sigma(x_12), x_12), Derivative(sigma(x_13), x_13)],
[Derivative(sigma(x_21), x_21), Derivative(sigma(x_22), x_22), Derivative(sigma(x_23), x_23)],
[Derivative(sigma(x_31), x_31), Derivative(sigma(x_32), x_32), Derivative(sigma(x_33), x_33)]])

In [44]:
sigmaApply_L(A*B).diff(B).subs(A,X).subs(B,W)#.doit()
### CANNOT go farther here because of noncommutative scalars in matrix error

Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]).T*Lambda(_d, Derivative(sigma(_d), _d)).(Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]])*Matrix([
[w_11, w_12],
[w_21, w_22],
[w_31, w_32]]))

In [45]:
#sigmaApply_L(X*W).subs(specToElemD).subs(elemToMatArgD).diff(B)#

In [46]:
sigmaApply_L(A*B).diff(B)#.replace(A,X).replace(B,W).replace(X*W,Nelem)

X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [47]:
sigmaApply_L(A*B).diff(B).subs({A*B : vN(A,B)})

X.T*Lambda(_d, Derivative(sigma(_d), _d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))

In [48]:
sigmaApply_L(A*B).diff(B).subs({A*B : vN(A,B)}).doit()

X.T*Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [49]:
sigmaApply_L(A*B).diff(B).subs({A*B : vN(A,B)}).subs(elemToMatArgD)

X.T*Lambda(_d, Derivative(sigma(_d), _d)).(Matrix([
[n_11(X, W), n_12(X, W)],
[n_21(X, W), n_22(X, W)],
[n_31(X, W), n_32(X, W)]]))

In [50]:
sigmaApply_L(A*B).diff(B).subs({A*B : vN(A,B)}).subs(elemToMatArgD).doit()

X.T*Matrix([
[Derivative(sigma(n_11(X, W)), n_11(X, W)), Derivative(sigma(n_12(X, W)), n_12(X, W))],
[Derivative(sigma(n_21(X, W)), n_21(X, W)), Derivative(sigma(n_22(X, W)), n_22(X, W))],
[Derivative(sigma(n_31(X, W)), n_31(X, W)), Derivative(sigma(n_32(X, W)), n_32(X, W))]])

In [51]:
sigmaApply_L(A*B).diff(B).subs({A*B : vN(A,B)}).doit()

X.T*Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [52]:
#part1=sigmaApply_L(A*B).diff(B).subs({A*B : vN(A,B)}).doit()

In [53]:
part1 = compose(sigmaApply, n)(A,B).replace(n, v).replace(sigmaApply, sigmaApply_).diff(B).subs({A*B : vN(A,B)}).doit()


part1.subs({A:X}) # replace won't work here

Matrix([
[x_11, x_12, x_13],
[x_21, x_22, x_23],
[x_31, x_32, x_33]]).T*Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [54]:
part1.subs({A:X}).doit()

Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

### COMPARE: Symbolic form vs. Direct form vs. Step by Step form (which goes from symbolic to direct form by replacing)
#### Symbolic Abstract Form (with respect to W):

In [55]:
Lc = compose(lambd, sigmaApply, n)(A, B)
symb = Lc.replace(n, v).replace(sigmaApply, sigmaApply_).diff(B)
symb

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

#### Direct form: (after the symbolic form)

In [56]:
Lc.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(W).subs(specToElemD)

Matrix([
[x_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_11*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_21*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_31*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32)],
[x_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_12*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_22*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_32*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_32)],
[x_13*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_11) + x_23*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_21) + x_33*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_31), x_13*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_12) + x_23*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, n_22) + x_33*

Just placing the "n" values right in place of the "epsilons" using the "doit" function:

In [57]:
direct = Lc.replace(n, vN).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_).subs(elemToSpecD).diff(W).subs(specToElemD).doit()
direct

Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

#### Step by Step Form:

In [58]:
assert symb == Lc.replace(n, v).replace(sigmaApply, sigmaApply_).diff(B)

symb.subs({A*B : vN(A,B)})#.doit()

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]]))

In [59]:
symb.subs({A*B : vN(A,B)}).doit() # the dummy variable for lambda still stays unapplied

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])))*X.T*Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [60]:
symb.subs({A*B : vN(A,B)}).subs({A:X}).doit() # two doits are equivalent to the last one at the end

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])))*Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

In [61]:
# Creating a special symbol just for the lambda_L function that has the appropriate shape after multiplying A*B. Supposed to represent a matrix R s.t. R == A * B (so that the indices after applying lambda_L are correct)
ABres = MatrixSymbol("R", A.shape[0], B.shape[1])
lambd_L = Lambda(ABres, sum(ABres))


symb.subs({A*B : vN(A,B)}).subs({A:X}).doit()#subs({lambd: lambd_L}).doit()

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Lambda(_d, sigma(_d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])))*Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

In [62]:
# yay finall got the dummy variable in the lambd to be applied!!
unapplied = sigmaApply_L(vN(A,B))
applied = unapplied.doit()

symb.subs({A*B : vN(A,B)}).subs({A:X}).doit().replace(unapplied, applied)

Subs(Derivative(lambda(_xi_1), _xi_1), _xi_1, Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))*Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

In [63]:
# THIS SEEMS WRONG : ??? how to tell for sure?
lambd(Selem).diff(Selem).replace(lambd, lambd_L).doit()

Matrix([
[Matrix([
[Matrix([
[1, 1],
[1, 1],
[1, 1]]), 0],
[                                0, 0],
[                                0, 0]]), Matrix([
[0, Matrix([
[1, 1],
[1, 1],
[1, 1]])],
[0,                                 0],
[0,                                 0]])],
[Matrix([
[                                0, 0],
[Matrix([
[1, 1],
[1, 1],
[1, 1]]), 0],
[                                0, 0]]), Matrix([
[0,                                 0],
[0, Matrix([
[1, 1],
[1, 1],
[1, 1]])],
[0,                                 0]])],
[Matrix([
[                                0, 0],
[                                0, 0],
[Matrix([
[1, 1],
[1, 1],
[1, 1]]), 0]]), Matrix([
[0,                                 0],
[0,                                 0],
[0, Matrix([
[1, 1],
[1, 1],
[1, 1]])]])]])

In [64]:
# THis seems right:
dL_dS = lambd(Selem).replace(lambd, lambd_L).diff(Selem)
dL_dS

Matrix([
[1, 1],
[1, 1],
[1, 1]])

In [65]:
lambd(Selem)

lambda(Matrix([
[sigma(n_11), sigma(n_12)],
[sigma(n_21), sigma(n_22)],
[sigma(n_31), sigma(n_32)]]))

In [66]:
# This is the X^T * dS/dN part
compose(sigmaApply, n)(A,B).replace(n, v).replace(sigmaApply, sigmaApply_).diff(B).subs({A*B : vN(A,B)}).doit()

X.T*Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [67]:
N = MatrixSymbol("N", A.shape[0], B.shape[1])
Matrix(N)

Matrix([
[N[0, 0], N[0, 1]],
[N[1, 0], N[1, 1]],
[N[2, 0], N[2, 1]]])

In [68]:
dS_dN = compose(sigmaApply)(N).replace(sigmaApply, sigmaApply_).diff(N).subs({N : vN(A,B)}).doit()
# WRONG:
#dS_dN = sigmaApply(Nelem).replace(sigmaApply, sigmaApply_).diff(Matrix(Nelem))
dS_dN

Matrix([
[Derivative(sigma(n_11), n_11), Derivative(sigma(n_12), n_12)],
[Derivative(sigma(n_21), n_21), Derivative(sigma(n_22), n_22)],
[Derivative(sigma(n_31), n_31), Derivative(sigma(n_32), n_32)]])

In [69]:
from sympy.physics.quantum import TensorProduct

#TensorProduct( X.T, dS_dN)


dN_dW = X.T
dS_dW = dN_dW * dS_dN
dS_dW
#HadamardProduct(X.T, dS_dN)

Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

In [70]:
from sympy import HadamardProduct

dL_dW = HadamardProduct(dS_dW , dL_dS)
dL_dW

HadamardProduct(Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]]), Matrix([
[1, 1],
[1, 1],
[1, 1]]))

One more time as complete symbolic form:
$$
\begin{aligned}
\frac{\partial L}{\partial W} &= \frac{\partial N}{\partial W} \times \frac{\partial S}{\partial N} \odot \frac{\partial L}{\partial S} \\
&= X^T \times  \frac{\partial S}{\partial N} \odot \frac{\partial L}{\partial S}
\end{aligned}
$$
where $\odot$ signifies the Hadamard product and $\times$ is matrix multiplication.

In [71]:
HadamardProduct(dN_dW * dS_dN, dL_dS)

HadamardProduct(Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]]), Matrix([
[1, 1],
[1, 1],
[1, 1]]))

In [72]:
direct

Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

In [73]:
assert HadamardProduct(dN_dW * dS_dN, dL_dS).equals(direct)

In [74]:
# too long to see in editor:
symb.subs({A*B : vN(A,B)}).subs({A:X}).doit().replace(lambd, lambd_L)

Subs(Derivative(_xi_1[0, 0] + _xi_1[0, 1] + _xi_1[1, 0] + _xi_1[1, 1] + _xi_1[2, 0] + _xi_1[2, 1], _xi_1), _xi_1, Lambda(_d, sigma(_d)).(Matrix([
[n_11, n_12],
[n_21, n_22],
[n_31, n_32]])))*Matrix([
[x_11*Derivative(sigma(n_11), n_11) + x_21*Derivative(sigma(n_21), n_21) + x_31*Derivative(sigma(n_31), n_31), x_11*Derivative(sigma(n_12), n_12) + x_21*Derivative(sigma(n_22), n_22) + x_31*Derivative(sigma(n_32), n_32)],
[x_12*Derivative(sigma(n_11), n_11) + x_22*Derivative(sigma(n_21), n_21) + x_32*Derivative(sigma(n_31), n_31), x_12*Derivative(sigma(n_12), n_12) + x_22*Derivative(sigma(n_22), n_22) + x_32*Derivative(sigma(n_32), n_32)],
[x_13*Derivative(sigma(n_11), n_11) + x_23*Derivative(sigma(n_21), n_21) + x_33*Derivative(sigma(n_31), n_31), x_13*Derivative(sigma(n_12), n_12) + x_23*Derivative(sigma(n_22), n_22) + x_33*Derivative(sigma(n_32), n_32)]])

In [75]:
symb.subs({lambd: lambd_L})

Subs(Derivative(_xi_1[0, 0] + _xi_1[0, 1] + _xi_1[1, 0] + _xi_1[1, 1] + _xi_1[2, 0] + _xi_1[2, 1], _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)

In [76]:
print(symb.subs({lambd: lambd_L}))

Subs(Derivative(_xi_1[0, 0] + _xi_1[0, 1] + _xi_1[1, 0] + _xi_1[1, 1] + _xi_1[2, 0] + _xi_1[2, 1], _xi_1), _xi_1, Lambda(_d, sigma(_d)).(X*W))*X.T*Lambda(_d, Derivative(sigma(_d), _d)).(X*W)


In [77]:
LcL = compose(lambd_L, sigmaApply, n)(A, B)
LcL

(sigma_apply(nu(X, W)))[0, 0] + (sigma_apply(nu(X, W)))[0, 1] + (sigma_apply(nu(X, W)))[1, 0] + (sigma_apply(nu(X, W)))[1, 1] + (sigma_apply(nu(X, W)))[2, 0] + (sigma_apply(nu(X, W)))[2, 1]

In [78]:
symbL = LcL.replace(n, v).replace(sigmaApply, sigmaApply_)#.diff(A)
symbL

Lambda(_d, sigma(_d)).(X*W)[0, 0] + Lambda(_d, sigma(_d)).(X*W)[0, 1] + Lambda(_d, sigma(_d)).(X*W)[1, 0] + Lambda(_d, sigma(_d)).(X*W)[1, 1] + Lambda(_d, sigma(_d)).(X*W)[2, 0] + Lambda(_d, sigma(_d)).(X*W)[2, 1]

In [79]:
compose(lambd, sigmaApply, n)(A,B).replace(n,v).subs({lambd:lambd_L})#.subs({sigmaApply : sigmaApply_L})

(sigma_apply(X*W))[0, 0] + (sigma_apply(X*W))[0, 1] + (sigma_apply(X*W))[1, 0] + (sigma_apply(X*W))[1, 1] + (sigma_apply(X*W))[2, 0] + (sigma_apply(X*W))[2, 1]

In [80]:
compose(lambd, sigmaApply, n)(A,B).replace(n,v).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_L)

Lambda(_d, sigma(_d)).(X*W)[0, 0] + Lambda(_d, sigma(_d)).(X*W)[0, 1] + Lambda(_d, sigma(_d)).(X*W)[1, 0] + Lambda(_d, sigma(_d)).(X*W)[1, 1] + Lambda(_d, sigma(_d)).(X*W)[2, 0] + Lambda(_d, sigma(_d)).(X*W)[2, 1]

In [81]:
compose(lambd, sigmaApply, n)(A,B).replace(n,v).replace(sigmaApply, sigmaApply_).replace(lambd, lambd_L).doit()

sigma(W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + sigma(W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + sigma(W[0, 0]*X[2, 0] + W[1, 0]*X[2, 1] + W[2, 0]*X[2, 2]) + sigma(W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2]) + sigma(W[0, 1]*X[1, 0] + W[1, 1]*X[1, 1] + W[2, 1]*X[1, 2]) + sigma(W[0, 1]*X[2, 0] + W[1, 1]*X[2, 1] + W[2, 1]*X[2, 2])

In [82]:
compose(lambd, sigmaApply, n)(A,B).replace(n,v).diff(B).doit()#replace(sigmaApply, sigmaApply_)#.replace(lambd, lambd_L).diff(B)

Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, X*W)*Derivative(lambda(sigma_apply(X*W)), sigma_apply(X*W))*Derivative(X*W, W)

In [83]:
compose(lambd, sigmaApply, n)(A,B).replace(n,v).diff(B).replace(lambd, lambd_L)

Derivative((sigma_apply(X*W))[0, 0] + (sigma_apply(X*W))[0, 1] + (sigma_apply(X*W))[1, 0] + (sigma_apply(X*W))[1, 1] + (sigma_apply(X*W))[2, 0] + (sigma_apply(X*W))[2, 1], sigma_apply(X*W))*Subs(Derivative(sigma_apply(_xi_1), _xi_1), _xi_1, X*W)*Derivative(X*W, W)

In [84]:
compose(lambd, sigmaApply, n)(A,B).replace(lambd, lambd_L)

(sigma_apply(nu(X, W)))[0, 0] + (sigma_apply(nu(X, W)))[0, 1] + (sigma_apply(nu(X, W)))[1, 0] + (sigma_apply(nu(X, W)))[1, 1] + (sigma_apply(nu(X, W)))[2, 0] + (sigma_apply(nu(X, W)))[2, 1]

In [85]:
compose(lambd, sigmaApply, n)(A,B).replace(lambd, lambd_L).replace(n, v).replace(sigmaApply, sigmaApply_)

Lambda(_d, sigma(_d)).(X*W)[0, 0] + Lambda(_d, sigma(_d)).(X*W)[0, 1] + Lambda(_d, sigma(_d)).(X*W)[1, 0] + Lambda(_d, sigma(_d)).(X*W)[1, 1] + Lambda(_d, sigma(_d)).(X*W)[2, 0] + Lambda(_d, sigma(_d)).(X*W)[2, 1]

In [86]:
compose(lambd, sigmaApply, n)(A,B).replace(lambd, lambd_L).replace(n, v).replace(sigmaApply, sigmaApply_).doit()#.diff(B)

sigma(W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + sigma(W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + sigma(W[0, 0]*X[2, 0] + W[1, 0]*X[2, 1] + W[2, 0]*X[2, 2]) + sigma(W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2]) + sigma(W[0, 1]*X[1, 0] + W[1, 1]*X[1, 1] + W[2, 1]*X[1, 2]) + sigma(W[0, 1]*X[2, 0] + W[1, 1]*X[2, 1] + W[2, 1]*X[2, 2])

In [87]:
compose(lambd, sigmaApply, n)(A,B).replace(lambd, lambd_L).replace(n, v).replace(sigmaApply, sigmaApply_).doit().diff(Matrix(B)).doit()

Matrix([
[X[0, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + X[1, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + X[2, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[2, 0] + W[1, 0]*X[2, 1] + W[2, 0]*X[2, 2]), X[0, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[0, 0] + W[1, 1]*X[0, 1] + W[2, 1]*X[0, 2]) + X[1, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[1, 0] + W[1, 1]*X[1, 1] + W[2, 1]*X[1, 2]) + X[2, 0]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 1]*X[2, 0] + W[1, 1]*X[2, 1] + W[2, 1]*X[2, 2])],
[X[0, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[0, 0] + W[1, 0]*X[0, 1] + W[2, 0]*X[0, 2]) + X[1, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[1, 0] + W[1, 0]*X[1, 1] + W[2, 0]*X[1, 2]) + X[2, 1]*Subs(Derivative(sigma(_xi_1), _xi_1), _xi_1, W[0, 0]*X[2, 0] + W[1, 0]*X[2, 1] + W[2, 0]*X[2, 2]), X[0, 1]*Subs(Deriva

In [88]:
sigmaApply = Function("sigma_apply", subscriptable=True)

#compose(lambd, sigmaApply, n)(A,B).replace(n,v).diff(B).replace(lambd, lambd_L).doit()# ERROR sigma apply is not subscriptable

#compose(lambd, sigmaApply, n)(A,B).replace(n,v).diff(B).subs({sigmaApply: sigmaApply_L})









In [89]:
compose(sigmaApply_L, sigmaApply_L)(A)

Lambda(_d, sigma(_d)).(Lambda(_d, sigma(_d)).(X))

In [90]:
x = Symbol('x', applyfunc=True)
#compose(sigmaApply_, sigmaApply_)(x)##ERROR
compose(sigmaApply_, sigmaApply_)(A)#.replace(A,f(x))

Lambda(_d, sigma(_d)).(Lambda(_d, sigma(_d)).(X))

In [91]:
compose(lambda_L, nL)(A,B)

0

In [92]:
n = Function("v", subscriptable=True) # doesn't work for below
#compose(lambda_L, n)(A,B).doit()

In [93]:
VL = Lambda((A,B), Lambda((A,B), MatrixSymbol("V", A.shape[0], B.shape[1])))
VL

Lambda((X, W), Lambda((X, W), V))

In [94]:
VL(A,B)

Lambda((X, W), V)

In [95]:
#saL = Lambda(A, Lambda(A, sigma(A)))
saL = Lambda(x, Lambda(x, sigma(x)))
#saL(n(A,B))## ERROR : the ultimate test failed: cannot even make this take an arbitrary function
#saL(n)
#s = lambda x : Lambda(x, sigma(x))
s = lambda x : sigma(x)
s(n(A,B))

sigma(v(X, W))

In [96]:
#sL = lambda x : Lambda(x, sigma(x))
#sL = Lambda(x, lambda x: sigma(x))

sL = Lambda(x, Lambda(x, sigma(x)))
sL
#sL(A)

Lambda(x, Lambda(x, sigma(x)))